## webscrape list of universities by country

In [1]:
#initial imports
from splinter import Browser
from bs4 import BeautifulSoup as soup
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import os
import pycountry

In [ ]:
# imports for closing pop ups
from selenium import webdriver
from selenium.webdriver.chrome.options import Options

In [ ]:
#ran into pop ups when automated browser clicks "next". 
# working around problem by opening and closing a new browser for each page of the website.


# create a list of urls to visit
urls = []

# add url for universities that start with 0-9 to list of urls
url = ("https://www.4icu.org/reviews/index0001.htm")
urls.append(url)
# print(urls)


# add urls for universities that start with A-Z to list of urls
#convert int to str found (https://stackoverflow.com/questions/52499991/how-to-append-a-integer-iteration-into-a-string-in-python)
# for url in urls:
for i in range(2, 28):       
    url = 'https://www.4icu.org/reviews/index'+ str(i) +'.htm'
    urls.append(url)
    
print(urls)

In [ ]:
#create loop to open new browser, visit each url in the urls list, extract the data, and close the browser

# Set up Splinter
executable_path = {'executable_path': ChromeDriverManager().install()}

#create an empty list for the table data
row_data = []

for i in urls:
    
    # set up browser
    browser = Browser('chrome', **executable_path, headless=False) 
               
    # Visit UniRank website
    url = i
    browser.visit(url)
    browser.is_element_present_by_css('div.list_text', wait_time=1)

    # parse the html
    html = browser.html
    html_soup = soup(html, 'html.parser')

    #isolate the table
    universities_table = html_soup.find('table', class_='table table-hover text-left')

    #loop through rows, collect data, append data to list, close browser
    for i in range(len(urls)):
        for row in universities_table.find_all('tr'):
            col = row.find_all('td')
            col = [ele.text.strip()for ele in col]
            row_data.append(col)
            
    # close browser          
    browser.quit()
        
# print(row_data)

In [ ]:
university_df = pd.DataFrame(row_data, columns=["University Name", "Country"])
university_df

In [ ]:
university_df.to_csv('C:/Users/sarah/OneDrive/Desktop/Global_GDP_Trends/Resources/webscraped_university_names.csv', index=True)

## ETL

In [2]:
file_to_load = pd.read_csv('Resources/webscraped_university_names.csv')

In [3]:
university_df = pd.DataFrame(file_to_load)
university_df

,Unnamed: 0,University Name,Country
0,0,NaN,NaN
1,1,3iL École d'ingénieurs,fr
2,2,NaN,NaN
3,3,3iL École d'ingénieurs,fr
4,4,NaN,NaN
...,...,...,...
375079,375079,ZNRF University of Management Sciences,bd
375080,375080,Zunyi Medical University,cn
375081,375081,Zunyi Normal College,cn
375082,375082,Zürcher Fachhochschule,ch


In [8]:
# university_df['Country Name']=university_df['Country Name'].country_alpha2_to_country_name(cn_name_format="default")


# def full_country_name(name):
#     for co in list(pycountry.countries):
        
# if name in co.name:
#     return




# use pycountry library to convert abbreviations to country names https://stackoverflow.com/questions/53923433/how-to-get-country-name-from-country-abbreviation-in-python-with-mix-of-alpha-2
list_alpha_2 = [i.alpha_2 for i in list(pycountry.countries)]

def standard_name(university_df):
    if (len(university_df['Country'])== "2 and university_df['Country'] in list_alpha_2):
        return pycountry.countries.get(alpha_2=university_df['Country']).name
    else:
        return'None'
    
university_df['Country Name']=university_df.apply(standard_name, axis = 1)
university_df

        
        # def standard_name(university_df):
#     if (university_df['Country'] in list_alpha_2):
#         return pycountry.countries.get(alpha_2=university_df['Country']).name
#     else:
#         return'None'
    


# duplicate row and replace values https://stackoverflow.com/questions/71200892/full-country-name-to-country-code-in-dataframe

,Unnamed: 0,University Name,Country,Country Name
0,0,NaN,NaN,None
1,1,3iL École d'ingénieurs,fr,None
2,2,NaN,NaN,None
3,3,3iL École d'ingénieurs,fr,None
4,4,NaN,NaN,None
...,...,...,...,...
375079,375079,ZNRF University of Management Sciences,bd,None
375080,375080,Zunyi Medical University,cn,None
375081,375081,Zunyi Normal College,cn,None
375082,375082,Zürcher Fachhochschule,ch,None
